# Prétraitement des données

## Suppression des éléments indésirables de nos tweets

In [326]:
# Imports nécessaires
import pandas as pd
import numpy as np
import string 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [38]:
# Lecture et stockage de la base de données
tweet_df = pd.read_pickle('../../delphes/data/final4_clean.csv')
tweet_df.head()

,mep_id,name,country,group,nat_group,twitter,content
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,[@danutahuebner Bardzo dziękuję @danutahuebner...
1,189525,Asim ADEMOV,Bulgaria,Group of the European People's Party (Christia...,Citizens for European Development of Bulgaria,AdemovAsim,[135 години единна и силна България. \nЧестит ...
2,124831,Isabella ADINOLFI,Italy,Non-attached Members,Movimento 5 Stelle,Isa_Adinolfi,"[Sembra un film, ma purtroppo è realtà: le imm..."
6,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,"[Clara Aguilera: ""El criterio científico debe ..."
7,204335,Alviina ALAMETSÄ,Finland,Group of the Greens/European Free Alliance,Vihreä liitto,alviinaalametsa,[@LHurttila @MariaOhisalo Kaupungilla on järje...


In [ ]:
age = []
sexe = []

In [85]:
y = np.arange(0, tweet_df.shape[0])
tweet_df['index'] = y
tweet_df.set_index('index', inplace=True)

In [87]:
new_test_df = pd.DataFrame(columns=['mep_id', 'name', 'country', 'group', 'nat_group', 'twitter', 'content'])

In [130]:
%%time
k = 0
for j in range(tweet_df.shape[0]):
    for i in range(len(tweet_df['content'][j])):
        new_test_df.loc[k] = tweet_df.loc[j,'mep_id':'twitter']
        new_test_df.loc[k, 'content'] = tweet_df['content'][j][i]
        k += 1

Wall time: 54min 54s


In [120]:
# Remove the undesirable elements in the entire dataframe
def rmurl_df(df, column_name):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
    return df



In [121]:
# Lowercase the tweet's column
def lower_df(df, column_name):
    '''
    This function lowercases a column made of strings.
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.lower()
    return df

In [122]:
# Remove the numbers in the tweet's column
def rmnumbers_df(df, column_name):
    '''
    This function removes all the digits of a column made of strings.
    '''
    df = df.copy()
    def remove_numbers(text):
        return ''.join(word for word in text if not word.isdigit())
    df[column_name] = df[column_name].apply(remove_numbers)
    return df

In [123]:
# Remove the undesirable punctuations in the tweet's column
def rmpunct_df(df, column_name):
    '''
    This function removes all the punctuations, all the "rt" and remove multiple spaces
    of a column made of strings.
    '''
    punct = string.punctuation
    df = df.copy()
    def replace_punct(text):
        for punctu in punct:
            text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
        return text
    df[column_name] = df[column_name].apply(replace_punct)
    return df

In [124]:
# Remove the stopwords in the tweet's column
def rmstopwords_df(df, column_name):
    '''
    This function removes all the stopwords of a column made of strings.
    '''
    df = df.copy()
    stop_words = stopwords.words('english')
    def remove_stopwords(text):
        for word in stop_words:
            text = text.replace(f' {word} ', ' ')
        return text
    df[column_name] = df[column_name].apply(remove_stopwords)
    return df

In [125]:
# Remove the undesirable emojis in the entire dataframe
def rmemojis_df(df):
    '''
    This function removes all the emojis of a column made of strings.
    Be careful to translate in latin alphabet before applying this function : 
    it also removes cyrillic alphabet
    '''
    df = df.copy()
    df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    return df

In [134]:
clean_df = rmurl_df(new_test_df, 'content')
clean_df = lower_df(clean_df, 'content')
clean_df = rmnumbers_df(clean_df, 'content')
clean_df = rmpunct_df(clean_df, 'content')
clean_df = rmstopwords_df(clean_df, 'content')
clean_df = rmemojis_df(clean_df)

In [138]:
clean_df[clean_df['country'] == 'Ireland'].head()

,mep_id,name,country,group,nat_group,twitter,content
24722,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the northern ireland protocol must protected l...
24723,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,as member special committee beating cancer loo...
24724,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this month shining light childrens cancer parl...
24725,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,yesterday told europe need keep speed amp infr...
24726,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this letter sent taoiseach clear seeking clari...


In [139]:
clean_df['content']

0         bardzo dzikuj niezalene od wadzy wolne media d...
1         komisja przyja arcywane projekty pilotaowe aut...
2         komisja przyja projekty pilotaowe mojego wspau...
3         w tym dniu w tym miejscu w tej godzinie prosz ...
4         bg nie potrzebuje by przez nikogo broniony nie...
                                ...                        
137295    tedenska akcija od etrtka srede zbornik bela k...
137296    strong amp way forward european future read eu...
137297    a date noted worrying state danger ends hu spe...
137298    ko vsi vedo da je nekaj noro pa se iz strahu p...
137299    tota ekipa tak se gre iz kranja z najlepim nav...
Name: content, Length: 137300, dtype: object

In [354]:
testbase_df = clean_df[clean_df['country'] == 'Ireland']

## Premier test Word2Vec

In [369]:
# Cette fonction retourne automatiquement X_train, X_test, y_train, y_test de notre base de données twitter.
def get_train_test_objects(df):
    '''
    Les étapes que cette fonction réalise sont en commentaires.
    '''
    # Copie de la base de données pour éviter les problèmes d'assignation abusive.
    df = df.copy() 
    # Récupération de tous les tweets et du nom du député qui les a posté. Création de la cible y.
    df = df[['name', 'content']]
    y = pd.get_dummies(df['name'])
    # Transformation des tweets en suite de mots (strings) dans une liste.
    sentences = df['content']
    sentences_inter = []
    for sentence in sentences:
        sentences_inter.append(sentence.split())
    # Séparation des données d'entraînement et de test
    sentences_train, sentences_test, y_train, y_test = train_test_split(sentences_inter, y, test_size = 0.3)
    # Vectorisation des phrases
    word2vec = Word2Vec(sentences=sentences_train)
    # Création des données d'entrée.
    X_train = embedding(word2vec,sentences_train)
    X_test = embedding(word2vec,sentences_test)
    X_train_pad = pad_sequences(X_train, padding='post',value=-1000, dtype='float32')
    X_test_pad = pad_sequences(X_test, padding='post',value=-1000, dtype='float32')
    # Création des données cibles.
    y_train = y_train.values
    y_test = y_test.values
    # Sorties de la fonction
    return X_train_pad, y_train, X_test_pad, y_test

In [370]:
def embed_sentence(word2vec, sentence):
    y = []
    for word in sentence:
        if word in word2vec.wv.vocab.keys():
           y.append(word2vec[word])
    return np.array(y)

def embedding(word2vec, sentences):
    
    y = []
    for sentence in sentences:
        y.append(embed_sentence(word2vec, sentence))
    return y

In [371]:
X_train, y_train, X_test, y_test = get_train_test_objects(testbase_df)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [375]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = -1000))
    model.add(layers.LSTM(15, activation='tanh'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])
    return model

model = init_model()

In [376]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 30, monitor='val_loss', restore_best_weights=True)
model = init_model()
model.fit(X_train, y_train, epochs=1000, validation_split = 0.3, callbacks = [es])

Train on 1470 samples, validate on 630 samples
Epoch 1/1000
1470/1470 [==============================] - 4s 3ms/sample - loss: 2.3020 - acc: 0.1082 - val_loss: 2.3031 - val_acc: 0.1016
Epoch 2/1000
1470/1470 [==============================] - 1s 780us/sample - loss: 2.2990 - acc: 0.1136 - val_loss: 2.3014 - val_acc: 0.1095
Epoch 3/1000
1470/1470 [==============================] - 1s 779us/sample - loss: 2.2961 - acc: 0.1388 - val_loss: 2.2996 - val_acc: 0.1206
Epoch 4/1000
1470/1470 [==============================] - 1s 762us/sample - loss: 2.2920 - acc: 0.1469 - val_loss: 2.2969 - val_acc: 0.1286
Epoch 5/1000
1470/1470 [==============================] - 1s 825us/sample - loss: 2.2865 - acc: 0.1537 - val_loss: 2.2968 - val_acc: 0.1159
Epoch 6/1000
1470/1470 [==============================] - 1s 754us/sample - loss: 2.2808 - acc: 0.1605 - val_loss: 2.2901 - val_acc: 0.1381
Epoch 7/1000
1470/1470 [==============================] - 1s 779us/sample - loss: 2.2745 - acc: 0.1707 - val_loss: 

1470/1470 [==============================] - 1s 830us/sample - loss: 2.1635 - acc: 0.1993 - val_loss: 2.2222 - val_acc: 0.1667
Epoch 59/1000
1470/1470 [==============================] - 1s 934us/sample - loss: 2.1600 - acc: 0.2054 - val_loss: 2.2353 - val_acc: 0.1540
Epoch 60/1000
1470/1470 [==============================] - 1s 1ms/sample - loss: 2.1561 - acc: 0.2041 - val_loss: 2.2401 - val_acc: 0.1651
Epoch 61/1000
1470/1470 [==============================] - 1s 915us/sample - loss: 2.1596 - acc: 0.2088 - val_loss: 2.2228 - val_acc: 0.1635
Epoch 62/1000
1470/1470 [==============================] - 1s 825us/sample - loss: 2.1583 - acc: 0.2136 - val_loss: 2.2287 - val_acc: 0.1683
Epoch 63/1000
1470/1470 [==============================] - 1s 801us/sample - loss: 2.1570 - acc: 0.2143 - val_loss: 2.2253 - val_acc: 0.1587
Epoch 64/1000
1470/1470 [==============================] - 1s 742us/sample - loss: 2.1578 - acc: 0.2014 - val_loss: 2.2299 - val_acc: 0.1698
Epoch 65/1000
1470/1470 [====

In [378]:
model.evaluate(X_test, y_test)

900/900 [==============================] - 0s 195us/sample - loss: 2.2116 - acc: 0.1789


[2.211647842195299, 0.17888889]

In [287]:
example = ['ireland should always belong to irish',
          'ireland should always belong to irish']
example_inter = []
for ex in example:
    example_inter.append(ex.split())
X_example = embedding(word2vec,example_inter)
X_example_pad = pad_sequences(X_example, padding='post',value=-1000, dtype='float32')

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [300]:
model.predict(X_example_pad)

array([[0.06270979, 0.15590073, 0.12469088, 0.05032336, 0.07766666,
        0.11065216, 0.15880199, 0.0959359 , 0.04862554, 0.11469299],
       [0.06270979, 0.15590073, 0.12469088, 0.05032336, 0.07766666,
        0.11065216, 0.15880199, 0.0959359 , 0.04862554, 0.11469299]],
      dtype=float32)